# Capstone Project - Segmenting and Clustering Neighborhoods in Toronto - Part 1

### Part 1 - Creation of Canada PostalCode DataFrame

a) Export canada postal code data from the wikipedia page to pandas dataframe

In [1]:
# Importing Basic Libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Importing libraries to query website and scrap it

from urllib.request import urlopen
from bs4 import BeautifulSoup

In [3]:
# Query the website and return the html to the variable 'webpage'

wiki_url = " https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
webpage = urlopen(wiki_url)

In [4]:
# Parse the html in the 'webpage' variable, and store it in Beautiful Soup format

soup = BeautifulSoup(webpage, 'lxml')
type(soup)

bs4.BeautifulSoup

Since there are more than one table in this wiki page, find the class of the required table to export data

In [5]:
# look at nested structure of HTML page
# print(soup.prettify())

# find all the tables in the webpage

data_table = soup.find_all('table')
# data_table --Commenting the line since the output is too large

# find the right table

right_table=soup.find('table', class_="wikitable sortable")
# right_table --Commenting the line since the output is too large

In [6]:
# Extract all table rows of the correct table and Print the last 3 rows for sanity check

rows = right_table.find_all('tr')
print(rows[-3:])

[<tr>
<td>M8Z</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td>Royal York South West
</td></tr>, <tr>
<td>M8Z</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td>South of Bloor
</td></tr>, <tr>
<td>M9Z</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>]


In [7]:
# iterate through table rows to form a list

row_list = []
for i,row in enumerate(rows):
    row_list.append(row.find_all('td'))
print(row_list[-3:])
# type(row_list)

[[<td>M8Z</td>, <td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>, <td>Royal York South West
</td>], [<td>M8Z</td>, <td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>, <td>South of Bloor
</td>], [<td>M9Z</td>, <td>Not assigned</td>, <td>Not assigned
</td>]]


In [8]:
# extract the text without html tags

row_list = []
for i,row in enumerate(rows):
    list1 = row.find_all('td')
    str_cells = str(list1)
    cleantext = BeautifulSoup(str_cells, "lxml").get_text()
    row_list.append(cleantext)

print(row_list[-3:])
# print(cleantext)

['[M8Z, Etobicoke, Royal York South West\n]', '[M8Z, Etobicoke, South of Bloor\n]', '[M9Z, Not assigned, Not assigned\n]']


In [9]:
# Convert list into a dataframe

toronto_df = pd.DataFrame(row_list)
toronto_df.head(10)

,0
0,[]
1,"[M1A, Not assigned, Not assigned\n]"
2,"[M2A, Not assigned, Not assigned\n]"
3,"[M3A, North York, Parkwoods\n]"
4,"[M4A, North York, Victoria Village\n]"
5,"[M5A, Downtown Toronto, Harbourfront\n]"
6,"[M5A, Downtown Toronto, Regent Park\n]"
7,"[M6A, North York, Lawrence Heights\n]"
8,"[M6A, North York, Lawrence Manor\n]"
9,"[M7A, Queen's Park, Not assigned\n]"


b) Clean and format the dataframe to get data in the required form

In [10]:
# Split and clean the df

# Drop first row
toronto_df.drop(toronto_df.index[[0]],inplace=True)
# Split the column into multiple columns
toronto_df1 = toronto_df[0].str.split(',', expand=True)
# Strip the '[,\n,]' characters from first and last columns
toronto_df1[0] = toronto_df1[0].str.strip('[')
toronto_df1[2] = toronto_df1[2].str.strip('\n]')
# Give column names
toronto_df1.columns = ['PostalCode','Borough','Neighbourhood']
# Strip any leading/trailing spaces from the values
toronto_df1['PostalCode'] = toronto_df1['PostalCode'].str.strip()
toronto_df1['Borough'] = toronto_df1['Borough'].str.strip()
toronto_df1['Neighbourhood'] = toronto_df1['Neighbourhood'].str.strip()
toronto_df1.head()

,PostalCode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


In [11]:
# Remove all rows with 'No assigned' value for Borough

# Rows before removal
print(toronto_df1.shape)

#Value counts
toronto_df1['Borough'].value_counts()

(288, 3)


Not assigned        77
Etobicoke           45
North York          38
Scarborough         37
Downtown Toronto    37
Central Toronto     17
West Toronto        13
York                 9
East Toronto         7
East York            6
Mississauga          1
Queen's Park         1
Name: Borough, dtype: int64

In [12]:
# Remove 'no assigned' rows

toronto_Df2 = toronto_df1[toronto_df1['Borough'] != 'Not assigned']
# reset index
toronto_Df2.reset_index(drop=True, inplace=True)
# Rows after removal
print(toronto_Df2.shape)
# See first five rows
toronto_Df2.head()

(211, 3)


,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [14]:
# Change the value of Neighborhood column to that of Borough column wherever no value is assigned e.g. Queen's Park Borugh(M7A)

toronto_Df2['Neighbourhood'] = toronto_Df2.apply(
    lambda row: row['Borough'] if row['Neighbourhood'] == 'Not assigned' else row['Neighbourhood'],
    axis=1
)
toronto_Df2.head(10)

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [15]:
# Group neighborhoods with same values for Postalcode and Borough

toronto_Df2 = toronto_Df2.groupby(['PostalCode','Borough'])['Neighbourhood'].apply(','.join).reset_index()
toronto_Df2.head(10)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [16]:
# Dimensions of the required dataframe (after joining)

toronto_Df2.shape

(103, 3)